In [1]:
from arango import ArangoClient

ArangoClient = ArangoClient(hosts="http://127.0.0.1:8530")

db = ArangoClient.db("yelp_db", username="super", password="grantaccess")

collections = db.collections()


In [6]:
import json

def preprocess_json(input_file, output_file):
    with open(input_file, 'r') as f:
        # Read the entire file content
        data = f.read()

        # Split the content by newline character to handle multiple JSON objects
        json_objects = data.strip().split('\n')

    # Process each JSON object separately
    processed_data = []
    for json_str in json_objects:
        try:
            # Load each JSON object separately
            obj = json.loads(json_str)
            processed_data.append(obj)
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)

    # Write processed data to the output file
    with open(output_file, 'w') as f:
        json.dump(processed_data, f)

if __name__ == "__main__":
    input_file = "yelp_academic_dataset_user.json"  # Replace with the path to your input JSON file
    output_file = "yelp_academic_dataset_user_updated.json"  # Replace with the path to your output JSON file
    preprocess_json(input_file, output_file)


In [7]:
import json
collection_name = "user"
collection=db.collection(collection_name)
# Path to the JSON file
json_file = "yelp_academic_dataset_user_updated.json"  

# Read the JSON data from file
with open(json_file, 'r') as f:
    data = json.load(f)

# Bulk insert the data into the collection
collection.import_bulk(data)

ConnectionAbortedError: Can't connect to host(s) within limit (3)

In [1]:
from arango import ArangoClient

# Connect to ArangoDB
client = ArangoClient(hosts='http://127.0.0.1:8530')
db = client.db('yelp_db', username='root', password='')



In [2]:
tipscollection=db.collection('tip')

cursor = tipscollection.all()

KeyboardInterrupt: 

In [3]:
# Your function to retrieve records and create edge documents
def process_records(cursor):
    edge_documents = []
    for record in cursor:
        # Assuming each record is a dictionary
        # Generate edge document for business
        edge_documents.append({
            "_from": f"tips/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"business/{record['business_id']}"
        })
    return edge_documents

# Process the cursor to generate edge documents
edge_documents = process_records(cursor)

In [4]:
print(len(edge_documents))

908915


In [3]:
# Function to insert records into an edge collection in batches
def insert_edges(collection_name, edge_documents, batch_size=1000):
    for i in range(0, len(edge_documents), batch_size):
        batch = edge_documents[i:i+batch_size]
        db[collection_name].import_bulk(batch)

# Split the edge documents into batches and insert them into the collection
# insert_edges('tip_business', edge_documents)


In [15]:
cursor = tipscollection.all()

In [16]:
def process_records_user(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"tips/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"user/{record['user_id']}"
        })
    return edge_documents


edge_documents_user = process_records_user(cursor)

In [17]:
print(len(edge_documents_user))

908915


In [18]:
insert_edges('tip_user', edge_documents_user)

In [2]:
checkinscollection=db.collection('checkin')
checkin_records = checkinscollection.all()

In [3]:
print(len(checkin_records))

131930


In [4]:
def process_records_checkin_business(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"checkin/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"business/{record['business_id']}"
        })
    return edge_documents

In [5]:
edge_documents_checkin = process_records_checkin_business(checkin_records)

474521
474522
474523
474524
474525
474526
474527
474528
474529
474530
474531
474532
474533
474534
474535
474536
474537
474538
474539
474540
474541
474542
474543
474544
474545
474546
474547
474548
474549
474550
474551
474552
474553
474554
474555
474556
474557
474558
474559
474560
474561
474562
474563
474564
474565
474566
474567
474568
474569
474570
474571
474572
474573
474574
474575
474576
474577
474578
474579
474580
474581
474582
474583
474584
474585
474586
474587
474588
474589
474590
474591
474592
474593
474594
474595
474596
474597
474598
474599
474600
474601
474602
474603
474604
474605
474606
474607
474608
474609
474610
474611
474612
474613
474614
474615
474616
474617
474618
474619
474620
474621
474622
474623
474624
474625
474626
474627
474628
474629
474630
474631
474632
474633
474634
474635
474636
474637
474638
474639
474640
474641
474642
474643
474644
474645
474646
474647
474648
474649
474650
474651
474652
474653
474654
474655
474656
474657
474658
474659
474660
474661
474662
474663

In [8]:
insert_edges('checkin_business', edge_documents_checkin)

In [4]:
def process_records_review_business(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"review/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"business/{record['business_id']}"
        })
    return edge_documents
def process_records_review_user(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"review/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"user/{record['user_id']}"
        })
    return edge_documents

In [5]:
reviewsCollection=db.collection('review')

review_records = reviewsCollection.all()


In [5]:
print(len(review_records))

6990280


In [17]:
type(review_records)

arango.cursor.Cursor

In [6]:
edge_documents_review_business = process_records_review_business(review_records)

In [7]:
insert_edges('review_business', edge_documents_review_business)

In [6]:
edge_documents_review_user = process_records_review_user(review_records)

In [7]:
insert_edges('review_user', edge_documents_review_user)